In [1]:
from pycocotools.coco import COCO
import os
print(os.getcwd())

/home/hope/R&D_Pro/HRNet


In [2]:
from pycocotools.coco import COCO
import os
print(os.getcwd())

val_coco = COCO("./data/bsf2023/annotations/coco_val.json")

# 看一下一个样本的 keypoints数量
ann_id = val_coco.getAnnIds()[0]
ann = val_coco.loadAnns(ann_id)[0]

print("关键点个数：", len(ann["keypoints"]) // 3)
print("关键点原始数据：", ann["keypoints"])


/home/hope/R&D_Pro/HRNet
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
关键点个数： 2
关键点原始数据： [2507.41, 1778.82, 2, 2560.35, 914.12, 2]


In [3]:
import pycocotools
print(pycocotools.__file__)

/home/hope/.conda-2024.02/envs/python3.7/lib/python3.7/site-packages/pycocotools/__init__.py


In [6]:
import os
import json
import cv2
import matplotlib.pyplot as plt

# 配置
coco_json_path = "./coco_val_test.json"     # 你的coco标注文件
image_root = "./data/bsf2023/train"    # 原图所在文件夹
save_dir = "./bbox_vis"                 # 可视化图片输出文件夹
draw_bbox = True                           # 是否在crop上画bbox（True画，False不画）

os.makedirs(save_dir, exist_ok=True)

# 加载 COCO 标注
with open(coco_json_path, 'r') as f:
    coco = json.load(f)

# 建立 image_id 到 file_name 的映射
id2name = {img["id"]: img["file_name"] for img in coco["images"]}

# 遍历每个标注
for ann_id, ann in enumerate(coco["annotations"]):
    image_id = ann["image_id"]
    bbox = ann["bbox"]  # [x_min, y_min, width, height]

    image_path = os.path.join(image_root, id2name[image_id])
    img = cv2.imread(image_path)

    if img is None:
        print(f"⚠️ Warning: {image_path} not found, skip.")
        continue

    # bbox 信息
    x, y, w, h = bbox
    x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)

    # 防止超出图像边界
    x1 = max(x1, 0)
    y1 = max(y1, 0)
    x2 = min(x2, img.shape[1] - 1)
    y2 = min(y2, img.shape[0] - 1)

    # 裁剪 bbox 区域
    crop = img[y1:y2, x1:x2]

    if draw_bbox:
        # 在 crop 上画一个小bbox边框，方便验证
        cv2.rectangle(crop, (0, 0), (crop.shape[1] - 1, crop.shape[0] - 1), (0, 255, 0), thickness=2)

    # 保存
    save_crop_path = os.path.join(save_dir, f"{id2name[image_id].split('.')[0]}_ann{ann_id}.jpg")
    cv2.imwrite(save_crop_path, crop)
    print(f"✅ Saved cropped image: {save_crop_path}")

print("\n🎯 所有bbox裁剪完成！请到 bbox_crops/ 目录查看。")

✅ Saved cropped image: ./bbox_vis/2023-07-10-15-12-29_ann0.jpg
✅ Saved cropped image: ./bbox_vis/2023-07-10-15-14-33_ann1.jpg

🎯 所有bbox裁剪完成！请到 bbox_crops/ 目录查看。
